Installing pyspark 

In [1]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 69kB/s 
     |████████████████████████████████| 204kB 38.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=3949d0f27d4e15a7733eda2e132cd66c733fc089eb3d926165b390b6827b4794
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


intializing Spark Session

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('RDD_Transformations').getOrCreate()

Mounting Google Drive


In [3]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


Referencing a dataset in an external storage system in Google Drive to create RDD

In [5]:
distFile = spark.sparkContext.textFile("/content/gdrive/My Drive/Colab Notebooks/data/dataset.txt")

RDD Creation

In [4]:
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
rdd = spark.sparkContext.parallelize(data,5)
rdd_collect = rdd.collect()
print("Number of Partitions: "+str(rdd.getNumPartitions()))
print("Action: First element: "+str(rdd.first()))
print(rdd_collect)


Number of Partitions: 5
Action: First element: 1
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


Action to read Data in RDD

In [6]:
for element in distFile.collect():
    print(element)

10	5	8	9	8
1	17	3	6	7
23	19	5	2	15
4	20	9	2	12
18	8	22	5	7


**flatMap(func)** is used When we want to output multiple elements from each input element we should use flatMap(func). 
According to the given dataset you have to output all numbers of each line by splitting from tab spaces and you can use the below code line to do it.


In [7]:
numbers = distFile.flatMap(lambda line: line.split("\t")) 
print(numbers.collect())

['10', '5', '8', '9', '8', '1', '17', '3', '6', '7', '23', '19', '5', '2', '15', '4', '20', '9', '2', '12', '18', '8', '22', '5', '7']


**map(func)** is used to return a new dataset formed by passing each element of the given dataset through a function.
Because the returned dataset is including string typed numbers you have to convert each element to integer. Therefore, using map(func) you should pass each element to int() function.


In [8]:
validNumbers = numbers.map(lambda number: int(number))
print(validNumbers.collect())

[10, 5, 8, 9, 8, 1, 17, 3, 6, 7, 23, 19, 5, 2, 15, 4, 20, 9, 2, 12, 18, 8, 22, 5, 7]


**filter(func)** is used to return a new dataset which is a subset of the given dataset. As mentioned above, you have to filter all numbers which are greater than 10. Therefore, you can use the below code line to do it.


In [9]:
filterNumber = validNumbers.filter(lambda number: number > 10) 
print(filterNumber.collect())

[17, 23, 19, 15, 20, 12, 18, 22]


**operations between two datasets**

Load dataset2 as RDD and apply flatMap tranformation

In [10]:
distFile2 = spark.sparkContext.textFile("/content/gdrive/My Drive/Colab Notebooks/data/dataset2.txt")
numbers2 = distFile2.flatMap(lambda line: line.split("\t"))

intersection(dataset2) function returns a new dataset which includes common elements of two datasets.


In [11]:
intersections = numbers.intersection(numbers2)
print(intersections.collect())

['10', '20', '15', '5']


To return a new dataset which includes all elements of given two elements, union(dataset2) function can be used.

In [12]:
unions = numbers.union(numbers2)
print(unions.collect())

['10', '5', '8', '9', '8', '1', '17', '3', '6', '7', '23', '19', '5', '2', '15', '4', '20', '9', '2', '12', '18', '8', '22', '5', '7', '10', '20', '30', '5', '10', '15', '15', '30', '45']


distinct() function is used to return a new dataset which contains distinct elements from a given dataset.
Let us take a new distinct dataset from the previously created ‘unions’ dataset.


In [13]:
distincts = unions.distinct() 
print(distincts.collect())

['10', '4', '20', '12', '3', '6', '7', '23', '15', '18', '30', '8', '9', '1', '17', '19', '22', '45', '5', '2']


**PairRDDFunctions** 

Even though Spark operations work on RDDs containing any type of objects, there are few transformation operations in Spark RDD which consider key-value pairs.To do those types of operations let us create a new dataset which contains (K,V) pairs from dataset.txt. We take each element dataset.txt as keys and assign 1 as value for each key.


In [14]:
pairs = validNumbers.map(lambda s: (s, 1))
print(pairs.collect())

[(10, 1), (5, 1), (8, 1), (9, 1), (8, 1), (1, 1), (17, 1), (3, 1), (6, 1), (7, 1), (23, 1), (19, 1), (5, 1), (2, 1), (15, 1), (4, 1), (20, 1), (9, 1), (2, 1), (12, 1), (18, 1), (8, 1), (22, 1), (5, 1), (7, 1)]


**groupByKey**() is used to return a dataset of (K,iterable<V>) pairs from a dataset of (K,V) pairs.Using the below code line you can return a new dataset of (K,iterable<V>) pairs grouping by keys. mapValues(list) is used to convert  iterable<V> as a list to visible elements of iterable<V>. 


In [15]:
groups = pairs.groupByKey().mapValues(list)
print(groups.collect())

[(10, [1]), (8, [1, 1, 1]), (6, [1]), (2, [1, 1]), (4, [1]), (20, [1]), (12, [1]), (18, [1]), (22, [1]), (5, [1, 1, 1]), (9, [1, 1]), (1, [1]), (17, [1]), (3, [1]), (7, [1, 1]), (23, [1]), (19, [1]), (15, [1])]


**ReduceByKey**(func) returns a dataset of (K,U) pairs from a dataset of (K,V) pairs. In this U is the aggregation of Vs which are related to the same keys.

In [16]:
counts = pairs.reduceByKey(lambda a, b: a + b)
print(counts.collect())

[(10, 1), (8, 3), (6, 1), (2, 2), (4, 1), (20, 1), (12, 1), (18, 1), (22, 1), (5, 3), (9, 2), (1, 1), (17, 1), (3, 1), (7, 2), (23, 1), (19, 1), (15, 1)]


**sortByKey**() returns a sorted dataset of (K,V) pairs from a dataset of (K,V) pairs. The returned dataset is sorted by keys in ascending or descending order.

In [19]:
sorts = counts.sortByKey(True)
print(sorts.collect())

[(1, 1), (2, 2), (3, 1), (4, 1), (5, 3), (6, 1), (7, 2), (8, 3), (9, 2), (10, 1), (12, 1), (15, 1), (17, 1), (18, 1), (19, 1), (20, 1), (22, 1), (23, 1)]
